
# UHC example
This example shows how to compute universal heart coordinates (UHC) for
the ventricles.


## Perform the required imports
Import the required modules and set relevant paths, including that of the working
directory, model, and ls-dyna executable (uses DEV-104373-g6d20c20aee).



In [ ]:
import copy
import os

import pyvista as pv

import ansys.heart.core.models as models
from ansys.heart.simulator.simulator import BaseSimulator, DynaSettings

# specify the path to the working directory and heart model
workdir = os.path.abspath(os.path.join("downloads", "Strocchi2020", "01", "FourChamber"))


path_to_model = os.path.join(workdir, "heart_model.vtu")

# specify LS-DYNA path
lsdyna_path = r"ls-dyna_smp"

# load heart model.
model: models.FourChamber = models.FourChamber(working_directory=workdir)
model.load_model_from_mesh(path_to_model, path_to_model.replace(".vtu", ".partinfo.json"))
model._extract_apex()
model.compute_left_ventricle_aha17()

## Instantiate the simulator object
instantiate simulator. Change options where necessary.



In [ ]:
# instantiate dyna settings of choice
dyna_settings = DynaSettings(
    lsdyna_path=lsdyna_path,
    dynatype="intelmpi",
    num_cpus=1,
)


simulator = BaseSimulator(
    model=model,
    dyna_settings=dyna_settings,
    simulation_directory=os.path.join(workdir, "simulation"),
)

## Compute UHC
Compute UHC using Laplace Dirichlet method.



In [ ]:
simulator.compute_uhc()

<div class="alert alert-info"><h4>Note</h4><p>There are several definitions for UHC (see https://github.com/KIT-IBT/Cobiveco).
   Here, a simple approach is taken and the
   Dirichlet conditions are shown below. At rotational direction, the start (pi), end (-pi)
   and middle (0) points are defined from four-cavity long axis cut view.</p></div>



<img src="file://_static/images/uvc_bc.png" width="600pt" align="center">



## Visualization of UVCs
UVC is assigned back to the full model automatically
Atrial points are padded with NaN's



In [ ]:
plotter = pv.Plotter(shape=(1, 3))

plotter.subplot(0, 0)
plotter.add_mesh(simulator.model.mesh, scalars="apico-basal")

plotter.subplot(0, 1)
plotter.add_mesh(copy.copy(simulator.model.mesh), scalars="transmural")

plotter.subplot(0, 2)
plotter.add_mesh(copy.copy(simulator.model.mesh), scalars="rotational")
plotter.show()

<img src="file://_static/images/uvc_result.png" width="600pt" align="center">

